In [16]:
# imports
import numpy as np
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

first_datetime = datetime.datetime.strptime('2018-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')

In [2]:
taxi_zone_lookup = pd.read_csv('nyc-tlc/misc/taxi _zone_lookup.csv')
print('taxi_zone_lookup:', taxi_zone_lookup.shape)
print(taxi_zone_lookup.head())
manhattan_location_ids = taxi_zone_lookup[taxi_zone_lookup['Borough']=='Manhattan']['LocationID'].values
print('manhattan_location_ids:', manhattan_location_ids.shape)
print(manhattan_location_ids)

taxi_zone_lookup: (265, 4)
   LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone
manhattan_location_ids: (69,)
[  4  12  13  24  41  42  43  45  48  50  68  74  75  79  87  88  90 100
 103 104 105 107 113 114 116 120 125 127 128 137 140 141 142 143 144 148
 151 152 153 158 161 162 163 164 166 170 186 194 202 209 211 224 229 230
 231 232 233 234 236 237 238 239 243 244 246 249 261 262 263]


In [17]:
def get_5min_id(x):
    return (x-first_datetime).seconds//(5*60)

def get_15min_id(x):
    return (x-first_datetime).seconds//(15*60)

def get_30min_id(x):
    return (x-first_datetime).seconds//(30*60)

In [20]:
sample = pd.read_csv('nyc-tlc/trip data/sample.csv')
print('sample:', sample.shape)
#print(sample.head())
sample_manhattan = sample[sample['PULocationID'].isin(manhattan_location_ids)].copy()
print('sample_manhattan:', sample_manhattan.shape)
#print(sample_manhattan.head())

sample_manhattan['store_and_fwd_flag'] = sample_manhattan['store_and_fwd_flag'].map(lambda x: x=='N' and 0 or 1)
sample_manhattan['tpep_pickup_datetime'] = pd.to_datetime(sample_manhattan['tpep_pickup_datetime'])
sample_manhattan['tpep_dropoff_datetime'] = pd.to_datetime(sample_manhattan['tpep_dropoff_datetime'])
sample_manhattan['trip_duration'] = sample_manhattan['tpep_dropoff_datetime']-sample_manhattan['tpep_pickup_datetime']
sample_manhattan['trip_duration'] = sample_manhattan['trip_duration'].map(lambda x: x.seconds)
sample_manhattan['trip_speed'] = sample_manhattan['trip_distance']/sample_manhattan['trip_duration']
sample_manhattan['tpep_pickup_year'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: x.year)
sample_manhattan['tpep_pickup_month'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: x.month)
sample_manhattan['tpep_pickup_day'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: x.day)
sample_manhattan['tpep_pickup_hour'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: x.hour)
sample_manhattan['tpep_pickup_weekday'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: x.weekday())
sample_manhattan['is_weekend'] = sample_manhattan['tpep_pickup_weekday'].map(lambda x: x >= 5 and 1 or 0)
sample_manhattan['is_morning_peak'] = sample_manhattan['tpep_pickup_hour'].map(lambda x: 7 <= x <= 9 and 1 or 0)
sample_manhattan['is_evening_peak'] = sample_manhattan['tpep_pickup_hour'].map(lambda x: 17 <= x <= 19 and 1 or 0)

sample_manhattan['tpep_pickup_5min_id'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: get_5min_id(x))
sample_manhattan['tpep_pickup_15min_id'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: get_15min_id(x))
sample_manhattan['tpep_pickup_30min_id'] = sample_manhattan['tpep_pickup_datetime'].map(lambda x: get_30min_id(x))

sample_manhattan.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

print(sample_manhattan.head())

sample: (1000, 17)
sample_manhattan: (908, 17)
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2018-01-01 00:21:05   2018-01-01 00:24:23                1   
1         1  2018-01-01 00:44:55   2018-01-01 01:03:05                1   
2         1  2018-01-01 00:08:26   2018-01-01 00:14:21                2   
3         1  2018-01-01 00:20:22   2018-01-01 00:52:51                1   
4         1  2018-01-01 00:09:18   2018-01-01 00:27:06                2   

   trip_distance  RatecodeID  store_and_fwd_flag  PULocationID  DOLocationID  \
0          0.500           1                   1            41            24   
1          2.700           1                   1           239           140   
2          0.800           1                   1           262           141   
3         10.200           1                   1           140           257   
4          2.500           1                   1           246           239   

   payment_type  fare

In [24]:
sample_5min = sample_manhattan.groupby(['tpep_pickup_5min_id', 'PULocationID']).mean()
print('sample_5min:', sample_5min.shape)
print(sample_5min.head())

sample_5min: (415, 26)
                                  VendorID  passenger_count  trip_distance  \
tpep_pickup_5min_id PULocationID                                             
0                   50               1.000            1.000          1.300   
                    74               2.000            1.000          0.470   
                    144              1.000            1.000          1.300   
                    148              1.500            2.000          4.125   
                    151              2.000            1.000          2.020   

                                  RatecodeID  store_and_fwd_flag  \
tpep_pickup_5min_id PULocationID                                   
0                   50                 1.000               1.000   
                    74                 1.000               1.000   
                    144                1.000               1.000   
                    148                1.000               1.000   
                    15

In [ ]:
tripdata_201801 = pd.read_csv('nyc-tlc/trip data/yellow_tripdata_2018-01.csv')
print('tripdata_201801:', tripdata_201801.shape, tripdata_201801.head())